## The Cross-Correlation Operation

In [2]:
import torch
from torch import nn

In [3]:
def corr2d(X,K):
    h,w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[0]):
            Y[i,j] = (X[i:i + h, j:j+w] * K).sum()
    return Y

In [4]:
X = torch.tensor([[0.0, 1.0, 2.0],[3.0, 4.0, 5.0],[6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0],[2.0, 3.0]])

In [5]:
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

## Convolutional Layers

In [6]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x,self.weight) + self.bias

In [7]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [8]:
K = torch.tensor([[1.0,-1.0]])

In [9]:
Y = corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

## Learning a Kernel

In [10]:
# Construct a two-dimensional convolutional layer with 1 output channel and a kernel of
# shape(1,2). For the sake of simoplicity, we ignore the bias here
conv2d = nn.LazyConv2d(1, kernel_size=(1, 2),bias=False)

# THe two-dimensional convolutional layer uses four-dimensional input and output in the format of (example, channel, height, width),
#  where the batch size (number of example in the batch) and the number of channels are both 1
X  = X.reshape((1, 1, 6, 8))
Y  = Y.reshape((1, 1, 6, 7))
lr = 3e-2 # learnnnig rate

In [11]:
for i in range(20):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # Update the kernel
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if(i + 1) % 2 == 0:
        print(f'epoch { i + 1}, loss {l.sum():.3f}')
    

epoch 2, loss 1.578
epoch 4, loss 0.363
epoch 6, loss 0.101
epoch 8, loss 0.033
epoch 10, loss 0.012
epoch 12, loss 0.005
epoch 14, loss 0.002
epoch 16, loss 0.001
epoch 18, loss 0.000
epoch 20, loss 0.000


In [12]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0011, -0.9987]])

## Exercises
1 Construct an image X with diagonal edges
    1.1 What happens if you apply the kernel K in section to it ?
    1.2 What happens if you transpose X ?
    1.3 What happens if you transpose K ?
2